In [25]:
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, utils
import matplotlib.pyplot as plt
from skimage import io, transform
from PIL import Image
import sklearn.metrics as mt
import os
from os import path
import copy
from importlib import reload
import itertools
import trainer as tr
import dataset as ds
import predict as pr
reload(tr)
reload(ds)
reload(pr)
# %matplotlib inline


<module 'predict' from '/Users/habbes/code/class/bd4h/chexpert/src/predict.py'>

In [26]:
os.chdir('..')

In [27]:
trainer = tr.Trainer(max_train_samples=1, value_for_uncertain=0.0)

Training using options arch densenet finetune False


/Users/habbes/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [28]:
ts = trainer.dataloaders['train'].dataset

In [34]:
ts[2]

(tensor([[[ 0.0056,  0.0056, -0.0116,  ..., -0.3027, -0.2684, -0.2513],
          [ 0.0569,  0.0227, -0.0116,  ..., -0.3027, -0.2856, -0.2856],
          [ 0.1254,  0.0569, -0.0116,  ..., -0.3027, -0.3198, -0.3369],
          ...,
          [-1.4329, -1.4329, -1.4329,  ..., -1.0562, -1.0733, -1.1247],
          [-1.4500, -1.4500, -1.4158,  ..., -1.0562, -1.0904, -1.1418],
          [-1.4500, -1.4500, -1.4158,  ..., -1.0562, -1.1075, -1.1589]],
 
         [[ 0.1352,  0.1352,  0.1176,  ..., -0.1800, -0.1450, -0.1275],
          [ 0.1877,  0.1527,  0.1176,  ..., -0.1800, -0.1625, -0.1625],
          [ 0.2577,  0.1877,  0.1176,  ..., -0.1800, -0.1975, -0.2150],
          ...,
          [-1.3354, -1.3354, -1.3354,  ..., -0.9503, -0.9678, -1.0203],
          [-1.3529, -1.3529, -1.3179,  ..., -0.9503, -0.9853, -1.0378],
          [-1.3529, -1.3529, -1.3179,  ..., -0.9503, -1.0028, -1.0553]],
 
         [[ 0.3568,  0.3568,  0.3393,  ...,  0.0431,  0.0779,  0.0953],
          [ 0.4091,  0.3742,

In [30]:
trainer2 = tr.Trainer(max_train_samples=1, value_for_uncertain=1.0)
ts2 = trainer2.dataloaders['train'].dataset

Training using options arch densenet finetune False


/Users/habbes/anaconda3/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [35]:
ts2[2]

(tensor([[[-2.0323, -1.6042, -0.9363,  ...,  2.0092,  1.9920,  1.9920],
          [-2.0323, -1.5870, -0.9020,  ...,  2.0263,  2.0092,  2.0092],
          [-2.0494, -1.5528, -0.8164,  ...,  2.0434,  2.0434,  2.0434],
          ...,
          [-0.9363, -0.9877, -1.0733,  ..., -0.8849, -0.9534, -0.9877],
          [-1.0390, -1.0733, -1.1075,  ..., -0.8678, -0.9192, -0.9363],
          [-1.0904, -1.1075, -1.1247,  ..., -0.8678, -0.9020, -0.9192]],
 
         [[-1.9482, -1.5105, -0.8277,  ...,  2.1835,  2.1660,  2.1660],
          [-1.9482, -1.4930, -0.7927,  ...,  2.2010,  2.1835,  2.1835],
          [-1.9657, -1.4580, -0.7052,  ...,  2.2185,  2.2185,  2.2185],
          ...,
          [-0.8277, -0.8803, -0.9678,  ..., -0.7752, -0.8452, -0.8803],
          [-0.9328, -0.9678, -1.0028,  ..., -0.7577, -0.8102, -0.8277],
          [-0.9853, -1.0028, -1.0203,  ..., -0.7577, -0.7927, -0.8102]],
 
         [[-1.7173, -1.2816, -0.6018,  ...,  2.3960,  2.3786,  2.3786],
          [-1.7173, -1.2641,

In [36]:
tdf = pd.read_csv(ds.TRAIN_CSV)

In [37]:
vdf = pd.read_csv(ds.VALID_CSV)

In [46]:
tdf['Path'][200]

'CheXpert-v1.0-small/train/patient00063/study5/view1_frontal.jpg'

In [42]:
vdf.head()['Path'][100]

KeyError: 100